In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [3]:
df = pd.read_csv('./data/ch6_p2.csv', header=None)
df.columns = ['weight']

In [16]:
stan_model = """
    data{
        int N;
        real X[N];
    }
    
    parameters{
        real<lower=0> sigma;
    }
    
    transformed parameters{
        real<lower=0> sigmasq;
        sigmasq <- pow(sigma, 2);
    }
    
    model{
        for(n in 1:N){
            X[n] ~ normal(130, sigma);
        }
    }
    
    generated quantities{
        real<lower=0, upper=1> sigmasq_over1;
        real<lower=0, upper=1> sigmasq_over2;
        
        sigmasq_over1 <- step( 1.0 - sigmasq);
        sigmasq_over2 <- step( 1.5 - sigmasq);
    }
"""

In [17]:
sm = pystan.StanModel(model_code = stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4a5bb49340c293ce4e7dbc5b3463deda NOW.


In [18]:
stan_data = {"N": df.shape[0], "X": df['weight']}

In [19]:
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

/Users/yugi/.pyenv/versions/anaconda-4.0.0/envs/py36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [20]:
fit

Inference for Stan model: anon_model_4a5bb49340c293ce4e7dbc5b3463deda.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
sigma           1.12  2.1e-3   0.19   0.82   0.98   1.09   1.23   1.58   8701    1.0
sigmasq          1.3  5.2e-3   0.48   0.67   0.97    1.2   1.51   2.49   8545    1.0
sigmasq_over1   0.28  4.2e-3   0.45    0.0    0.0    0.0    1.0    1.0  11554    1.0
sigmasq_over2   0.74  4.2e-3   0.44    0.0    0.0    1.0    1.0    1.0  10901    1.0
lp__          -11.42  6.6e-3   0.72 -13.46 -11.59 -11.14 -10.97 -10.91  11980    1.0

Samples were drawn using NUTS at Mon Aug 13 23:11:52 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).